In [2]:
import numpy as np

In [12]:
def initParameters(layerDims):
    np.random.seed(3)
    parameter={}
    L = len(layerDims)
    for l in range(1, L):
        parameter["W" + str(l)] = np.random.randn(layerDims[l], layerDims[l-1])
        parameter["b" + str(l)] = np.zeros((layerDims[l],1))
    return parameter

In [4]:
def sigmoid(Z):
    A = 1/(1 + np.exp(-Z))
    cache = Z
    return A, cache

In [5]:
def relu(Z):
    A = np.maximum(0,Z)
    cache = Z
    return A, cache

In [8]:
def Dsigmoid(dA, cache):
    Z= cache
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)
    return dZ

In [20]:
def dRelu(dA,cache):
    Z = cache
    dZ = np.array(dA, copy = True)
    dZ[dZ <= 0] = 0
    return dZ

In [9]:
def forward(A,W,b):
    Z = np.dot(W,A) + b
    cache = (A,W,b)
    return Z, cache

In [10]:
def actForward(Aprev, W, b,act):
    Z, linearCache = forward(Aprev, W, b)
    if act == "sigmoid":
        A, actCache = sigmoid(Z)
    elif act == "relu":
        A, actCache = relu(Z)
    cache = (linearCache, actCache)
    return A, cache

In [18]:
def DeepForward(X, parameters):
    caches = []
    A = X
    L = len(parameters)
    for l in range(1, L):
        Aprev = A
        A, cache = actForward(A, parameters["W" + str(l)],parameters["b" + str(l)], "relu" )
        caches.append(cache)
    AL, cache = actForward(A, parameters["W" + str(l)],parameters["b" + str(l)], "sigmoid" )
    caches.append(cache)
    return AL, caches

In [17]:
def computeCost(AL,Y):
    m = Y.shape[1]
    cost = -np.sum(np.dot(y, np.log(AL)) + np.dot((1-Y), np.log(1-AL)))/m
    cost= np.squeeze(cost)
    return cost

In [19]:
def backward(dZ,cache):
    Aprev, W, b = cache 
    m = Aprev.shape[1]
    dW = np.dot(dZ, Aprev)/m
    db = np.sum(dZ, axis = 1, keepdims=True)/m
    dAprev = np.dot(W.T, dZ)
    return dAprev, dW, db

In [21]:
def actBackward(dA, cache, act):
    linearCache, actCache = cache
    if act == "relu":
        dZ = dRelu(dA, actCache)
        dAprev, dW, db = backward(dZ, linearCache)
    if act == "sigmoid":
        dZ = Dsigmoid(dA, actCache)
        dAprev, dW, db = backward(dZ, linearCache)
    return dAprev, dW, db

In [22]:
def deepBackward(AL, Y, cache):
    grads = {}
    L = len(cache)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)
    dAL = -(np.divide(Y, AL) - np.divide(1-Y, 1-AL))
    currentCache = cache[L-1]
    dAprevTemp, dWTemp, dbTemp = actBackward(dAL, currentCache, "sigmoid")
    grads["dA" + str(L-1)] = dAprevTemp
    grads["dW" + str(L)] = dWTemp
    grads["db" + str(L)] = dbTemp
    return grads

In [23]:
def updateParameters(parameters, grads, learningRate):
    parameters = copy.deepcopy(parameters)
    L = len(parameters)//2
    for l in range(L):
        parameters["W" + str(l)] -= learningRate*grads["dW" + str(l+1)]
        parameters["b" + str(l)] -= learningRate*grads["db" + str(l+1)]
    return parameters

In [24]:
def predic(X,Y, parameters):
    m = X.shape[1]
    n = len(Y)
    p = np.zeros((1,m))
    proba,caches = DeepForward(X, parameters)
    for l in range(0, proba.shape[1]): 
        if proba[0,l] > 0.5:
            proba[0,l] = 1
        else :
            proba[0,l] = 0
    return p 

In [27]:
def LLayerModel(X,Y, layerDims, learningRate  =0.0075, iter = 3000, printCost =False):
    np.random.seed(1)
    cost = []
    parameters = initParameters (layerDims)
    for l in range(0,iter):
        AL, caches = DeepForward(X, parameters)
        cost = computeCost(AL, Y)
        grads = deepBackward(AL,Y,caches)
        parameters = updateParameters(parameters, grads, learningRate)
        if printCost and i % 100 == 0 or i == iter-1:
            print("The cost function {}: {}".format(i, np.squeeze(cost)))
        if i % 100 == 0 or i == iter:
            cost.squeeze(cost)
    return parameters, cost

I still need to learn how to use the data and start training. Dr Erwin hero?